# LLMCompiler
LLMCompiler is an agent architecture to speed up the execution of agentic tasks by eagerly-executed tasks within a DAG. It also saves costs on redundant token usage by reducing the number of calls to the LLM.
<br>
It has 3 main components:
1. Planner: stream a DAG of tasks
2. Task Fetching Unit: Schedules and executes the tasks as soon as they are executable
3. Joiner: Responds to the user or triggers a second plan

In [5]:
import re
import os
import json
import base64
import asyncio
import platform
import requests
import operator
import playwright
import numpy as np
import pandas as pd
import datetime as dt

from enum import Enum
from typing import List
from typing import Dict
from typing import Tuple
from typing import Union
from typing import Literal
from typing import Optional
from typing import Sequence
from typing import Annotated
from typing import TypedDict
from operator import itemgetter

from IPython import display
from IPython.display import HTML
from IPython.display import Image

from langgraph.graph import END
from langgraph.graph import StateGraph

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from langchain_core.tools import BaseTool
from langchain_core.messages import BaseMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.chat import ChatMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage
from langchain_core.messages.function import FunctionMessage
from langchain_core.prompts.image import ImagePromptTemplate

from langchain_core.pydantic_v1 import Field
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableBranch
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

from langchain_core.runnables.graph import CurveStyle
from langchain_core.runnables.graph import NodeColors
from langchain_core.runnables.graph import MermaidDrawMethod

from langchain_core.language_models import BaseChatModel

from langchain import hub
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.agents import create_openai_functions_agent
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults

from math_tools import get_math_tool
from output_parser import LLMCompilerPlanParser, Task

from dotenv import load_dotenv
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'LLMCompiler'

## 1. Tools
We'll first define the tools for the agent to use in our demo. We'll give it the class search engine + calculator combo

In [2]:
calculate = get_math_tool(ChatOpenAI(model='gpt-4o'))
search = TavilySearchResults(max_results=1, description='tavily_search_results_json(query="the search query") - a search engine.')
tools = [search, calculate]

/Users/aman/anaconda3/envs/pinegap/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` that is available on ChatModels capable of tool calling. You can read more about the method here: https://python.langchain.com/docs/modules/model_io/chat/structured_output/ Please follow our extraction use case documentation for more guidelines on how to do information extraction with LLMs. https://python.langchain.com/docs/use_cases/extraction/. If you notice other issues, please provide feedback here: https://github.com/langchain-ai/langchain/discussions/18154
  warn_deprecated(


In [3]:
calculate.invoke({
    'problem': 'What is the temp of sf + 57',
    'context': ['The temperature of sf is 32 degrees']
})

'89'

## 2. Planner
Largely adopted from the original source code, the planner accepts the input question and generates a task list to execute.<br>
If it is provided with a previous plan, it is instructed to re-plan, which is useful if, upon completion of the first batch of tasks, the agent must take more actions.<br><br>
The code below constructs the prompt template for the planner and composes it with LLM and output parser, defined in output_parser.py. The output parser processes a task list in the following form.<br>
```plaintext
1. tool_1(arg1='arg1', arg2=3.5, ...)
Thought: I then want to find out Y by using tool_2
2. tool_2(arg1='', arg2='${1}')
3. join()<END_OF_PLAN>
```

The Thought lines are optional. The ${#} placeholders are variables. These are used to route tool (task) outputs to other tools.

In [6]:
prompt = hub.pull('wfh/llm-compiler')
print(prompt.pretty_print())

================================ System Message ================================

Given a user query, create a plan to solve it with the utmost parallelizability. Each plan should comprise an action from the following {num_tools} types:
{tool_descriptions}
{num_tools}. join(): Collects and combines results from prior actions.

 - An LLM agent is called upon invoking join() to either finalize the user query or wait until the plans are executed.
 - join should always be the last action in the plan, and will be called in two scenarios:
   (a) if the answer can be determined by gathering the outputs from tasks to generate the final response.
   (b) if the answer cannot be determined in the planning phase before you execute the plans. Guidelines:
 - Each action described above contains input/output types and description.
    - You must strictly adhere to the input and output types for each action.
    - The action descriptions contain the guidelines. You MUST strictly follow those guideline

In [7]:
def create_planner(llm: BaseChatModel, tools: Sequence[BaseTool], base_prompt: ChatPromptTemplate):
    tool_descriptions = '\n'.join(f"{i + 1}. {tool.description}\n" for i, tool in enumerate(tools))
    planner_prompt = base_prompt.partial(replan='', num_tools=len(tools) + 1, tool_descriptions=tool_descriptions)
    replanner_prompt = base_prompt.partial(
        replan=' - You are given "Previous Plan" which is the plan that the previous agent created along with the execution results '
        '(given as Observation) of each plan and a general thought (given as Thought) about the executed results.'
        'You MUST use these information to create the next plan under "Current Plan".\n'
        ' - When starting the Current Plan, you should start with "Thought" that outlines the strategy for the next plan.\n'
        ' - In the Current Plan, you should NEVER repeat the actions that are already executed in the Previous Plan.\n'
        ' - You must continue the task index from the end of the previous one. Do not repeat task indices.',
        num_tools=len(tools) + 1,
        tool_descriptions=tool_descriptions
    )

    def should_replan(state: list):
        # context is passed as a system message
        return isinstance(state[-1], SystemMessage)
    
    def wrap_messages(state: list):
        return {'messages': state}
    
    def wrap_and_get_last_index(state: list):
        next_task = 0
        for message in state[::-1]:
            if isinstance(message, FunctionMessage):
                next_task = message.additional_kwargs['idx'] + 1
                break
        state[-1].content = state[-1].content + f' - Begin counting at : {next_task}'
        return {'messages': state}
    
    return (
        RunnableBranch((should_replan, wrap_and_get_last_index | replanner_prompt), wrap_messages | planner_prompt)
        | llm
        | LLMCompilerPlanParser(tools=tools)
    )

In [8]:
llm = ChatOpenAI(model='gpt-4o')
planner = create_planner(llm, tools, prompt)

In [9]:
example_question = 'What is the temperature in SF raised to the 3rd power?'

for task in planner.stream([HumanMessage(content=example_question)]):
    print(task['tool'], task['args'])
    print('----')

description='tavily_search_results_json(query="the search query") - a search engine.' max_results=1 {'query': 'current temperature in San Francisco'}
----
name='math' description='math(problem: str, context: Optional[list[str]]) -> float:\n - Solves the provided math problem\n - `problem` can either be a simple math problem (eg "1 + 3") or a word problem (eg "how many apples are there if there are 3 apples and 2 apples").\n - You cannot calculate multiple expressions in one call. For instance, `math("1 + 3, 2 + 4")` does not work. If you need to calculate multiple expressions, you need to call them separately like `math("1 + 3")` and then `math("2 + 4")`\n - Minimize the number of `math` actions as much as possible. For instance, instead of calling 2. math("what is the 10% of $1") and then call 3. math("$1 + $2"), you MUST call 2. math("what is the 110% of $1") instead, which will reduce the number of math actions.\n - You can optionally provide a list of strings as `context` to help t